In [2]:
import pandas as pd
import numpy as np
import joblib

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from feature_engine.imputation import MeanMedianImputer, CategoricalImputer
from feature_engine.encoding import CountFrequencyEncoder
from feature_engine.transformation import LogTransformer
from feature_engine.selection import DropFeatures

# Cargamos operadores definidos por desarrollador
#import utils.operators as utils_ops
import operators

In [3]:
#!pip install feature-engine

# 1.- Cargamos Datos de entrenamiento

In [4]:
data_train = pd.read_csv("../data/raw/train.csv")
data_train['MSSubClass'] = data_train['MSSubClass'].astype('O')
data_train['GarageCars'] = data_train['GarageCars'].astype('O')
data_train['BsmtFullBath'] = data_train['BsmtFullBath'].astype('O')
data_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# 2.- Train, TEst Split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data_train.drop(["Id", "SalePrice"], axis = 1), data_train["SalePrice"],test_size=0.3, random_state=2025)

# 3.- Configuración del pipeline

In [6]:
# imputación de variables categóricas
CATEGORICAL_VARS_WITH_NA_FREQUENT = ['BsmtQual', 'BsmtExposure','BsmtFinType1','GarageFinish','BsmtFullBath','Functional',
                                     'GarageCars','MSZoning','Exterior1st','KitchenQual']

# imputación de variagbles numéricas con imputación por media
NUMERICAL_VARS_WITH_NA = ['LotFrontage','GarageArea']

#imputación de variables categoricas con indicador de Faltante (Missing)
CATEGORICAL_VARS_WITH_NA_MISSING = ['FireplaceQu']

#Variables temporales
TEMPORAL_VARS = ['YearRemodAdd']

#Año de Referencia
REF_VAR = "YrSold"

#Variables para Binarización por sesgo
BINARIZE_VARS = ['ScreenPorch']

#Variables que eliminaremos
DROP_FEATURES = ["YrSold"]

#Variables para transfomraicón logarítmica
NUMERICAL_LOG_VARS = ["LotFrontage", "1stFlrSF", "GrLivArea"]

#Variables para codificación ordinal.
QUAL_VARS = ['ExterQual', 'BsmtQual', 'HeatingQC', 'KitchenQual', 'FireplaceQu']

#variables especiales
EXPOSURE_VARS = ['BsmtExposure']
FINISH_VARS = ['BsmtFinType1']
GARAGE_VARS = ['GarageFinish']
FENCE_VARS = ['Fence']

#Variables para codificación por frecuencia (no ordinal)
CATEGORICAL_VARS = ['MSZoning',  'LotShape',  'LandContour',
                    'LotConfig', 'Neighborhood', 'RoofStyle', 'Exterior1st',
                    'Foundation', 'CentralAir', 'Functional', 'PavedDrive',
                    'SaleCondition']

#Mapeo para varibels categótricas para calidad.
QUAL_MAPPINGS = {'Po': 1, 'Fa': 2, 'TA': 3,
                 'Gd': 4, 'Ex': 5, 'Missing': 0, 'NA': 0}

EXPOSURE_MAPPINGS = {'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4}

FINISH_MAPPINGS = {'Missing': 0, 'NA': 0, 'Unf': 1,
                   'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}

GARAGE_MAPPINGS = {'Missing': 0, 'NA': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3}
                   
                   #Variables a utilzar en el entrenamiento

FEATURES = [
    'MSSubClass',
    'MSZoning',
    'LotFrontage',
    'LotShape',
    'LandContour',
    'LotConfig',
    'Neighborhood',
    'RoofStyle',
    'Exterior1st',
    'ExterQual',
    'Foundation',
    'BsmtQual',
    'BsmtExposure',
    'BsmtFinType1',
    'HeatingQC',
    'CentralAir',
    '1stFlrSF',
    'GrLivArea',
    'BsmtFullBath',
    'KitchenQual',
    'Functional',
    'FireplaceQu',
    'GarageFinish',
    'GarageCars',
    'GarageArea',
    'PavedDrive',
    'WoodDeckSF',
    'SaleCondition'
]


# 4.- Seleccion de features de interes

In [ ]:
#X_train = X_train[FEATURES]
#X_test = X_test[FEATURES]

In [12]:
ALL_FEATURES = set(X_train.columns)
FEATURES_TO_DROP = ALL_FEATURES.difference(FEATURES)
FEATURES_TO_DROP = list(FEATURES_TO_DROP)

# 5.- Construccion del pipeline

In [13]:
house_prices_data_proc = Pipeline([
    #0. Selección de Features para Modelo
    ('drop_features',
        DropFeatures(features_to_drop=FEATURES)),

    #1. imputación de variables categóricas
    ('cat_missing_imputation',
        CategoricalImputer(imputation_method='missing', variables=CATEGORICAL_VARS_WITH_NA_MISSING)),

    #2. Imputación de variables categoricas por frecuencia
    ('cat_missing_freq_imputation',
        CategoricalImputer(imputation_method='frequent', variables=CATEGORICAL_VARS_WITH_NA_FREQUENT)),

    #3. Imputación de variables numéricas
    ('mean_imputation',
        MeanMedianImputer(imputation_method='mean',variables=NUMERICAL_VARS_WITH_NA)),

    #4. Codificación de Variables Categóricas
    ('quality_mapper',
        #operators.Mapper(variables = CATEGORICAL_VARS, mappins=QUAL_MAPPINGS)),
        operators.Mapper(variables=QUAL_VARS, mappins=QUAL_MAPPINGS)),
    ('exposure_mapper',
        operators.Mapper(variables=EXPOSURE_VARS, mappins= EXPOSURE_MAPPINGS)),
    ('garage_mapper',
        operators.Mapper(variables = GARAGE_VARS, mappins=GARAGE_MAPPINGS)),
    ('finish_mapper',
        operators.Mapper(variables=FINISH_VARS, mappins = FINISH_MAPPINGS)),

    # 5 Codificación por frequency encoding
    ('cat_freq_encode',
        CountFrequencyEncoder(encoding_method='count', variables=CATEGORICAL_VARS)),
    
    #6. Transformación de variables continuas
    ('continues_log_transform',
        LogTransformer(variables=NUMERICAL_LOG_VARS)),

    #7. Normalización de variables
    ('variable_scaler',
        MinMaxScaler())
])

# 6. Aplicamos Pipeline para construcción de features

In [14]:
house_prices_data_proc.fit(X_train, y_train)

ValueError: The resulting dataframe will have no columns after dropping all existing variables

In [ ]:
def save_procesed_data(X,y,str_df_name):
    X = X[FEATURES]
    X_transformed = house_prices_data_proc.transform(X)
    df_X_transformed = pd.DataFrame(data = X_transformed, columns=FEATURES)
    y = y.reset_index()
    df_transformed = pd.concat([df_X_transformed, y['SalePrice']],axis=1)
    df_transformed.to_csv(f"../data/interim/proc_{str_df_name}.csv",index = False)

: 

### Procesamiento data de Entrenamiento

In [ ]:
save_procesed_data(X_train, y_train, str_df_name = "data_train")

: 

### Procesamiento data de entrenamiento

In [ ]:
save_procesed_data(X_test, y_test, str_df_name = "data_test")

c:\Users\DC\anaconda3\envs\venv_papd_a\lib\site-packages\feature_engine\encoding\base_encoder.py:260: UserWarning: During the encoding, NaN values were introduced in the feature(s) Exterior1st, Functional.
  warnings.warn(


: 

# 7.- Exportamos Pipeline de Pre-Procesamiento ya configurado

In [ ]:
joblib.dump(house_prices_data_proc, '../models/house_prices_pre_proc_pipeline.pkl')

['../models/house_prices_pre_proc_pipeline.pkl']

: 

## 1. Que